## Importing the libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
%matplotlib inline

## Reading in the data files

In [ ]:
df_calendar = pd.read_csv('/Users/rwalters/Documents/Airbnb Data 2020/calendar.csv')
df_listings = pd.read_csv('/Users/rwalters/Documents/Airbnb Data 2020/listings.csv', dtype='str')
df_reviews = pd.read_csv('/Users/rwalters/Documents/Airbnb Data 2020/reviews.csv')

## Exploring the datasets

### Listings

In [ ]:
df_listings.shape

In [ ]:
# show listings data sample
df_listings.head()

In [ ]:
df_listings.isnull().sum()

### Reviews

In [ ]:
df_reviews.shape

In [ ]:
# show reviews data sample
df_reviews.head()

In [ ]:
df_reviews.isnull().sum()

### Calendar

In [ ]:
df_calendar.shape

In [ ]:
# show calendar data sample
df_calendar.head()

In [ ]:
df_calendar.dtypes

## The business questions

**Question 1**

- What types of holidays are people interested in? 
    - Can a recommendation be created from the reviews?

**Question 2**
- What features assist in obtaining a good review score?

**Question 3**
- Is there a seasonal trend to the availability of accommodation?

In [ ]:
df_calendar.isnull().sum()

In [ ]:
df_calendar = df_calendar.fillna(0)

In [ ]:
def change_to_float(df):
    df = df.str.replace(',', '')
    return df.str.replace('$', '').astype('float64')

In [ ]:
def change_to_int(df):
    return df.astype('int64')

In [ ]:
df_calendar['price'] = change_to_float(df_calendar['price'])
df_calendar['adjusted_price'] = change_to_float(df_calendar['adjusted_price'])

In [ ]:
df_calendar['minimum_nights'] = change_to_int(df_calendar['minimum_nights'])
df_calendar['maximum_nights'] = change_to_int(df_calendar['maximum_nights'])

In [ ]:
df_calendar['date'] = df_calendar['date'].astype('datetime64[ns]')

In [ ]:
df_calendar.dtypes

In [ ]:
df_calendar.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

### Preparing the data for displaying

In [ ]:
date_count_df = df_calendar.groupby('date').size().reset_index(name='count')

In [ ]:
date_count_df.head()

In [ ]:
plt.plot_date(date_count_df['date'], date_count_df['count'])
plt.show()

**Question 4**
- What is the average price for the different location? what would affect this price? 

**Question 5**
- What type of accommodation is most popular? i.e. number of rooms availble